In [1]:
# -*- coding: utf-8 -*-
import argparse
import copy
import json
import os
import pickle
import random
import re
import sys
from collections import defaultdict, deque
from typing import List
import re
from typing import Any, List
import ipydagred3
import networkx as nx
import numpy as np
import openai
from tqdm import tqdm


from SCAN_TOT_utils import *

#os.environ["http_proxy"] = "http://localhost:7890"
#os.environ["https_proxy"] = "http://localhost:7890"
setOpenAi(keyid = 4)

GPT_MODEL = "gpt-4-turbo"
GPT_MODEL = "gpt-4o-mini"
GPT_MODEL = "gpt-3.5-turbo"



In [2]:
def split_instruction(instruction):
    parts = instruction.split('OUT:')
    question = parts[0].replace('IN:', '').strip()
    answer = parts[1].strip()
    return question, answer

In [3]:
tasks = []
solutions = []
count = 0
N =100
with open("/Users/natehu/Desktop/Tsinghua Research/TaDe/v1/scan_tasks.txt", 'r', encoding= 'utf-8') as file:
    for line in file:            
            question, actions = split_instruction(line.strip())            
            tasks.append(question)
            actions = [action.replace("I_", "") for action in actions.split()]
            solutions.append(actions)
            if count == N:
                break
    

In [46]:
i = 8

question = tasks[i]
golden_answer = solutions[i]

print(question)
print(golden_answer)

jump left twice after turn around right twice
['TURN_RIGHT', 'TURN_RIGHT', 'TURN_RIGHT', 'TURN_RIGHT', 'TURN_RIGHT', 'TURN_RIGHT', 'TURN_RIGHT', 'TURN_RIGHT', 'TURN_LEFT', 'JUMP', 'TURN_LEFT', 'JUMP']


In [47]:

decompose_steps = decompose_sql(question)
steps, steps_dict = convert_steps_to_format(decompose_steps)
num_steps = len(steps)
print(steps)
print(steps_dict)  # 只是加了一个问题的编号而已.

total tokens: 706
['jump left', 'jump left twice', 'turn right', 'turn around right', 'turn around right twice']
{1: 'jump left', 2: 'jump left twice', 3: 'turn right', 4: 'turn around right', 5: 'turn around right twice'}


In [48]:
# TOT 求解
N = 3  # 每个子问题进行N次proposal
M = 2  # 通过评估选出M个最好的proposal

solution = []

for i in range(num_steps):
    subtask = steps[i]
    sys_q = f"""

    There is a natural language instruction representing a sequence of actions. I need you to translate this sentence from natural language into a standardized meta-action sequence."
    Here is the instruction:\n{question}

    I have broken this instruction down into some smaller instructions. I will assign you sub-instructions one by one, and provide the results of the previous sub-instructions as a reference for your reasoning.
    Please organize your reasoning according to the combination and progression of actions.

    For your reference, 13 examples for translation together with the corresponding explanations are as follows:

    Q: "turn left"
    A: "turn left" outputs "TURN LEFT".

    Q: "turn right"
    A: "turn right" outputs "TURN RIGHT".

    Q: "jump left"
    A: The output of “jump left” concatenates: the output of “turn left”, the output of “jump”. “turn left” outputs “TURN LEFT”. “jump” outputs “JUMP”. So concatenating the output of “turn left” and the output of “jump” leads to “TURN LEFT” + “JUMP”. So the output of “jump left” is “TURN LEFT” + “JUMP”.

    Q: "run right"
    A: The output of "run right" concatenates: the output of "turn right", the output of "run". "turn right" outputs "TURN RIGHT". "run" outputs "RUN". So concatenating the output of "turn right" and the output of "run" leads to "TURN RIGHT" + "RUN". So the output of "run right" is "TURN RIGHT" + "RUN".

    Q: "look twice"
    A: The output of "look twice" concatenates: the output of "look", the output of "look". "look" outputs "LOOK". So repeating the output of "look" two times leads to "LOOK" * 2. So the output of "look twice" is "LOOK" * 2.

    Q: "run and look twice"
    A: The output of "run and look twice" concate+nates: the output of "run", the output of "look twice". "run" outputs "RUN". "look twice" outputs "LOOK" * 2. So concatenating the output of "run" and the output of "look twice" leads to "RUN" + "LOOK" * 2. So the output of "run and look twice" is "RUN" + "LOOK" * 2.

    Q: "jump right thrice"
    A: The output of "jump right thrice" concatenates: the output of "jump right", the output of "jump right", the output of "jump right". "jump right" outputs "TURN RIGHT" + "JUMP". So repeating the output of "jump right" three times leads to ("TURN RIGHT" + "JUMP") * 3. So the output of "jump right thrice" is ("TURN RIGHT" + "JUMP") * 3.

    Q: "walk after run"
    A: The output of "walk after run" concatenates: the output of "run", the output of "walk". "run" outputs "RUN". "walk" outputs "WALK". So concatenating the output of "run" and the output of "walk" leads to "RUN" + "WALK". So the output of "walk after run" is "RUN" + "WALK".

    Q: "turn opposite left"
    A: The output of "turn opposite left" concatenates: the output of "turn left", the output of "turn left". "turn left" outputs "TURN LEFT". So repeating the output of "turn left" twice leads to "TURN LEFT" * 2. So the output of "turn opposite left" is "TURN LEFT" * 2.

    Q: "turn around left"
    A: The output of "turn around left" concatenates: the output of "turn left", the output of "turn left", the output of "turn left", the output of "turn left". "turn left" outputs "TURN LEFT". So repeating the output of "turn left" four times leads to "TURN LEFT" * 4. So the output of "turn around left" is "TURN LEFT" * 4. Q: "turn opposite right" A: The output of "turn opposite right" concatenates: the output of "turn right", the output of "turn right". "turn right" outputs "TURN RIGHT". So repeating the output of "turn right" twice leads to "TURN RIGHT" * 2. So the output of "turn opposite right" is "TURN RIGHT" * 2.

    Q: "turn around right"
    A: The output of "turn around right" concatenates: the output of "turn right", the output of "turn right", the output of "turn right", the output of "turn right". "turn right" outputs "TURN RIGHT". So repeating the output of "turn right" four times leads to "TURN RIGHT" * 4. So the output of "turn around right" is "TURN RIGHT" * 4.

    Q: "walk opposite left"
    A: The output of "walk opposite left" concatenates: the output of "turn opposite left", the output of "walk". "turn opposite left" outputs "TURN LEFT" * 2. "walk" outputs "WALK". So concatenating the output of "turn opposite left" and the output of "walk" leads to "TURN LEFT" * 2 + "WALK". So the output of "walk opposite left" is "TURN LEFT" * 2 + "WALK".

    Q: "walk around left"
    A: The output of "walk around left" concatenates: the output of "walk left", the output of "walk left", the output of "walk left", the output of "walk left". "walk left" outputs "TURN LEFT" + "WALK". So repeating the output of "walk around left" four times leads to ("TURN LEFT" + "WALK") * 4. So the output of "walk around left" is ("TURN LEFT" + "WALK") * 4.

    Please pay attention to the use of parentheses.
"""  

    subask = f"""\nThe sub-instruction to be converted now is:: {subtask}
Based on the information above, please provide a concise and clear answer"""
    
    if len(solution)==0:
        # 第一个子问题
        query = subask
        Q = [{'role':'system', 'content':sys_q},
            {'role':'user', 'content':query},]
        for n in range(N):  # 一个子问题提问N次,获取N个解
            result = askChatGPT(Q, model=GPT_MODEL, temperature=1)
            eval_Q = Q + [{'role':'assistant', 'content':result}]
            eval_Q = eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this solution, on a scale from 1 to 5. Only output the number."}]
            score = askChatGPT(eval_Q, model=GPT_MODEL, temperature=1)
            score = int(score)
            
            solution.append((score, [result]))  # 维护一整条推理路径
            
        solution = sorted(solution, key=lambda x: x[0])
        solution = solution[:M]  # 剪枝
    else:
        temp_solution = []
        for m in range(M):  # 因为剪枝动态维护M个推理路径
            answersSoFar = f"""\nSo far, the answers to the preceding sub-instructions are as follows: The format is Sub-problem-Id: xxx; Sub-problem: xxx; Answer: xxx."""
            for index, value in enumerate(solution[m][1]):
                try:
                    answersSoFar += f"""\nSub-problem-Id: {index}; Sub-problem: {steps[index]}; Answer: {value}."""
                except:
                    print('warning')
                    print(index)
                    print(len(solution[m][1]))
                    print(len(steps))
                    sys.exit(0)
            query = answersSoFar+subask
            Q = [{'role':'system', 'content':sys_q},
                 {'role':'user', 'content':query},]
            for n in range(N):  # 一个子问题提问N次,获取N个解
                result = askChatGPT(Q, model=GPT_MODEL, temperature=1)
                eval_Q = Q + [{'role':'assistant', 'content':result}]
                eval_Q = eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this solution, on a scale from 1 to 5. Only output the number."}]
                score = askChatGPT(eval_Q, model=GPT_MODEL, temperature=1)
                score = int(score)
                
                temp_solution.append((solution[m][0]+score, solution[m][1]+[result]))  # 路径score累加
        
        # print(len(temp_solution))  # 此时temp_solution中应该有M*N种推理路径
        solution = sorted(temp_solution, key=lambda x: x[0])
        solution = solution[:M]  # 剪枝 M*N->M

print(len(solution))
printSeq(solution)
# 用额外的一次query再问一下最终的答案


total tokens: 1380
total tokens: 1416
total tokens: 1383
total tokens: 1419
total tokens: 1383
total tokens: 1419
total tokens: 1468
total tokens: 1504
total tokens: 1470
total tokens: 1506
total tokens: 1468
total tokens: 1504
total tokens: 1471
total tokens: 1507
total tokens: 1471
total tokens: 1507
total tokens: 1471
total tokens: 1507
total tokens: 1508
total tokens: 1544
total tokens: 1508
total tokens: 1544
total tokens: 1508
total tokens: 1544
total tokens: 1515
total tokens: 1551
total tokens: 1510
total tokens: 1546
total tokens: 1512
total tokens: 1548
total tokens: 1555
total tokens: 1591
total tokens: 1555
total tokens: 1591
total tokens: 1555
total tokens: 1591
total tokens: 1555
total tokens: 1591
total tokens: 1555
total tokens: 1591
total tokens: 1555
total tokens: 1591
total tokens: 1607
total tokens: 1643
total tokens: 1607
total tokens: 1643
total tokens: 1607
total tokens: 1643
total tokens: 1607
total tokens: 1643
total tokens: 1607
total tokens: 1643
total tokens

In [49]:
solution

[(24,
  ['"jump left" outputs "TURN LEFT" + "JUMP".',
   'Sub-problem-Id: 1; Sub-problem: jump left twice; Answer: ("TURN LEFT" + "JUMP") * 2.',
   'Sub-problem-Id: 2; Sub-problem: turn right; Answer: "TURN RIGHT".',
   'Sub-problem-Id: 3; Sub-problem: turn around right; Answer: "TURN RIGHT" * 4.',
   'Sub-problem-Id: 4; Sub-problem: turn around right twice; Answer: ("TURN RIGHT" * 4) * 2.']),
 (24,
  ['"jump left" outputs "TURN LEFT" + "JUMP".',
   'Sub-problem-Id: 1; Sub-problem: jump left twice; Answer: ("TURN LEFT" + "JUMP") * 2.',
   'Sub-problem-Id: 2; Sub-problem: turn right; Answer: "TURN RIGHT".',
   'Sub-problem-Id: 3; Sub-problem: turn around right; Answer: "TURN RIGHT" * 4.',
   'Sub-problem-Id: 4; Sub-problem: turn around right twice; Answer: ("TURN RIGHT" * 4) * 2.'])]

In [60]:
user_q = f"""There is a natural language command representing a sequence of actions:\n{question}

I have broken this action down into a series of smaller pieces and each sub-problem is solved.
The sub-problems and their corresponding answers are as follows. """

for index, value in enumerate(solution[0][1]):  # 这里仅仅使用了最终得分最高的1条路径来总结得到final answer
    user_q += f"""\nSub-problem-Id: {index}; Sub-problem: {steps[index]}; Answer: {value}."""

user_q += "\n Given the results from the sub problems and their answer, so what is the final answer? (Try to concatenate some of the sub-problems  answers to get the final answer.)"

final_Q = [{'role':'user', 'content':user_q}]


print(final_Q)

[{'role': 'user', 'content': 'There is a natural language command representing a sequence of actions:\njump left twice after turn around right twice\n\nI have broken this action down into a series of smaller pieces and each sub-problem is solved.\nThe sub-problems and their corresponding answers are as follows. \nSub-problem-Id: 0; Sub-problem: jump left; Answer: "jump left" outputs "TURN LEFT" + "JUMP"..\nSub-problem-Id: 1; Sub-problem: jump left twice; Answer: Sub-problem-Id: 1; Sub-problem: jump left twice; Answer: ("TURN LEFT" + "JUMP") * 2..\nSub-problem-Id: 2; Sub-problem: turn right; Answer: Sub-problem-Id: 2; Sub-problem: turn right; Answer: "TURN RIGHT"..\nSub-problem-Id: 3; Sub-problem: turn around right; Answer: Sub-problem-Id: 3; Sub-problem: turn around right; Answer: "TURN RIGHT" * 4..\nSub-problem-Id: 4; Sub-problem: turn around right twice; Answer: Sub-problem-Id: 4; Sub-problem: turn around right twice; Answer: ("TURN RIGHT" * 4) * 2..\n Given the results from the sub 

In [61]:
finalResult = askChatGPT(final_Q, model=GPT_MODEL, temperature=1)

total tokens: 350


In [62]:
question

'jump left twice after turn around right twice'

In [63]:
finalResult

'The final answer is: ("TURN RIGHT" * 4) * 2 + ("TURN LEFT" + "JUMP") * 2.'

In [64]:
actionSeq = sentenceRes2Actions(finalResult)

total tokens: 568


In [65]:
actionSeq

'TURN_RIGHT TURN_RIGHT TURN_RIGHT TURN_RIGHT TURN_RIGHT TURN_RIGHT TURN_RIGHT TURN_RIGHT TURN_LEFT JUMP TURN_LEFT JUMP'

In [66]:
actionList = actionSeq.split()

In [67]:
actionList

['TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_LEFT',
 'JUMP',
 'TURN_LEFT',
 'JUMP']

In [68]:
golden_answer

['TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_RIGHT',
 'TURN_LEFT',
 'JUMP',
 'TURN_LEFT',
 'JUMP']

In [70]:
actionList == golden_answer

True

In [72]:
actionList = []

In [73]:
if actionList == golden_answer:
    
    print('correct')
    
elif actionList != golden_answer:
    
    print('incorrect')
    

incorrect


### wode

In [ ]:
# TOT 第二种求解方式

N = 3  # 先进性N次任务分解，然后 N次做题，最后评估


# 任务分解阶段
decompositions = []
for i in range(N):
    plan, Q = decompose_sql(question, type)
    result, result_dict = convert_steps_to_format(plan)
    
    eval_Q = Q + [{'role':'assistant', 'content':f"""The plan is : {result}"""}]
    eval_Q = eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this step to step plan, on a scale from 1 to 10. Only output the number."}]
    printSeq(eval_Q)
    score = askChatGPT(eval_Q, model=GPT_MODEL, temperature=1)
    score = int(score)
    
    decompositions.append((score, result_dict))

# 选择最佳任务分解方案
task_decompositions = sorted(decompositions, key=lambda x: x[0], reverse=True)
best_task_decomposition = task_decompositions[0][1] 

print("******************************解题阶段******************************")
# 解题阶段
solutions = []
for i in range(N):
    question_input = f"""
There is a math_problem. I need you to solve it and give an answer.
Here is the problem:\n{question}

I have broken this math problem down into a series of smaller problems. Here is the list of sub-problems: {best_task_decomposition}
Please solve the problem and respond according to mathematical logic.
    
    """
    query = {"role": "user", "content": question_input}
    Q_jie = [query]
    sol = askChatGPT(Q_jie, model=GPT_MODEL, temperature=1)
    sol_eval_Q = Q_jie + [{'role':'assistant', 'content':sol}]
    sol_eval_Q = sol_eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this solution, on a scale from 1 to 10. Only output the number."}]
    printSeq(sol_eval_Q)
    sol_score = askChatGPT(sol_eval_Q, model=GPT_MODEL, temperature=1)
    sol_score = int(sol_score)
    
    solutions.append((sol_score, sol))

# 选择最佳解答
sub_solutions = sorted(solutions, key=lambda x: x[0], reverse=True)
best_solution = sub_solutions[0][1]  # 选择最好的子问题解

print("best_solution: ", best_solution)

print("******************************评估结果******************************")

last_query = [{
    "role": "user",
    "content": f""" 
    Given the problem and step-by-step plan and the solution as following: {best_solution}
    
    What is the final answer?
    Please give the final answer without any additional explanation or clarification."""
}]

finalResult = askChatGPT(last_query, model=GPT_MODEL, temperature=1)
print(finalResult)


# 让大语言模型来判断有没有回答正确
judgeAnswer = {'role':'user', 'content':f"""Here is a math problem with a standard answer and a student's solution. Please help me determine if the student's solution is correct. If the numerical value are same, then it is correct.

Problem: {question}

Standard answer: {gold_answer}

Answer: {finalResult}

If the student's answer is correct, just output True; otherwise, just output False.
No explanation is required.
"""}

Q_judge = [judgeAnswer]
ifcorrect = askChatGPT(Q_judge, model=GPT_MODEL, temperature=1)  # 要么是True, 要么是False

if 'True' in ifcorrect:
    print('correct')
    success = True  # 任务未受中断,完整地结束了,所以标记为成功
elif 'False' in ifcorrect:
    print('error')
    success = True  # 任务未受中断,完整地结束了,所以标记为成功                       
